## IPython + jit decorator

Let's say that we have this kind of code:

In [1]:
import numpy as np

In [2]:
image = np.random.rand(1024, 1024)

In [3]:
def laplace(image):
    """Laplace operator in NumPy for 2D images."""
    laplacian = (
        image[:-2, 1:-1] + image[2:, 1:-1] + image[1:-1, :-2] + image[1:-1, 2:]
        - 4*image[1:-1, 1:-1]
    )
    thresh = np.abs(laplacian) > 0.05
    return thresh

We can define a jitted function in IPython (and Jupyter, which uses IPython). The only limitation is that everything needed for the function (even `np`) has to be (re)defined in the same cell!

In [4]:
import numpy as np
from transonic import jit

@jit
def laplace_pythran(image):
    """Laplace operator in NumPy for 2D images."""
    laplacian = (
        image[:-2, 1:-1] + image[2:, 1:-1] + image[1:-1, :-2] + image[1:-1, 2:]
        - 4*image[1:-1, 1:-1]
    )
    thresh = np.abs(laplacian) > 0.05
    return thresh

We first call the function to launch the compilation ("warmup the jit"):

In [5]:
result = laplace_pythran(image)

Schedule pythranization: laplace_pythran.py


Since the compilation is quit long, we explicitelly wait for the extension to benchmark the compiled version.

In [6]:
from transonic import wait_for_all_extensions
wait_for_all_extensions()

Wait for all extensions


Pythranizing /home/pierre/.transonic/pythran/__jit__/__ipython__25970b035d85f309561968e246899d70/laplace_pythran.py
File /home/pierre/.transonic/pythran/__jit__/__ipython__25970b035d85f309561968e246899d70/laplace_pythran_0e47d9b355b188b1feabdbf18fdee11f_8b9e99482f5d1fa6279790dd20e989ce.cpython-38-x86_64-linux-gnu.so created by pythran
Pythran stdout:
running build_ext
new_compiler returns <class 'distutils.unixccompiler.UnixCCompiler'>
building 'laplace_pythran_0e47d9b355b188b1feabdbf18fdee11f_8b9e99482f5d1fa6279790dd20e989ce' extension
C compiler: clang++-10 -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC

creating /tmp/tmprbjl4zn8/tmp
compile options: '-DENABLE_PYTHON_MODULE -D__PYTHRAN__=3 -DPYTHRAN_BLAS_BLAS -I/home/pierre/.virtualenvs/env_transonic/lib/python3.8/site-packages/pythran 

Pythran stderr:
INFO:      sys file exists: /home/pierre/.virtualenvs/env_transonic/lib/python3.8/site-packages/pythran/pythran.cfg
INFO: platform file exists: /home/pierre/.virtualenvs/env_transonic/lib/python3.8/site-packages/pythran/pythran-linux.cfg
INFO:     user file exists: /home/pierre/.pythranrc
INFO: pythranrc section [pythran] is valid and options are correct
INFO: pythranrc section [typing] is valid and options are correct
INFO: pythranrc section [compiler] is valid and options are correct
INFO: Generated module: laplace_pythran_0e47d9b355b188b1feabdbf18fdee11f_8b9e99482f5d1fa6279790dd20e989ce
INFO: Output: laplace_pythran_0e47d9b355b188b1feabdbf18fdee11f_8b9e99482f5d1fa6279790dd20e989ce.tmp



Now, let's benchmark the 2 functions:

In [7]:
from transonic.util import timeit_verbose

namespace = {"laplace": laplace, "laplace_pythran": laplace_pythran, "image": image}

norm = timeit_verbose("laplace(image)", globals=namespace)
time_pythran = timeit_verbose("laplace_pythran(image)", globals=namespace, norm=norm)
print(f"\nIt corresponds to a speedup of ~ {norm/time_pythran:.1f}!")

laplace                          :     1 * norm
norm = 0.0101 s
laplace_pythran                  : 0.116 * norm

It corresponds to a speedup of ~ 8.6!
